In [ ]:
!pip install -r requirements.txt # run this cell to install all the packages needed 

In [2]:
#import of libraries 

import os
import webbrowser
import s3fs
import matplotlib


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cftime
import plotly.graph_objects as go
import calendar

import contextily as ctx
import folium
from folium import LayerControl
import geopandas as gpd

from shapely.geometry import Polygon, LineString, Point
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import xarray as xr
import netCDF4 as nc
from netCDF4 import Dataset
from IPython.display import display

In [3]:
# Creation of two folders to centralize data and results
data_dir = "./data"  
os.makedirs(data_dir, exist_ok=True) 
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)


# Access information to the cloud MinIO Client (Eve's bucket)
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "esam"